In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.datasets import load_digits

In [2]:
dataset = pd.read_csv('/Users/pawankumarkc/Documents/vscode_workspace/MLAlgo/datasets/Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
dataset['Exited'].value_counts()

#Imbalanced data

0    7963
1    2037
Name: Exited, dtype: int64

In [4]:
dataset = dataset.iloc[:,3:]
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
dataset.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [7]:
dataset = pd.get_dummies(dataset, columns=['Geography','Gender'], drop_first=True)

In [8]:
dataset.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [9]:
## Split the data into dep and ind variable

x = dataset.drop(['Exited'], axis=1) 
y = dataset[['Exited']]

In [10]:
# Balance the data

from imblearn.over_sampling import SMOTE
smote = SMOTE()
x_smote, y_smote = smote.fit_resample(x, y)
print(y.value_counts())
print(y_smote.value_counts())

Exited
0         7963
1         2037
dtype: int64
Exited
0         7963
1         7963
dtype: int64


In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_smote, y_smote, test_size=0.25, random_state=101)

In [12]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [13]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout, BatchNormalization

In [14]:
x.shape

(10000, 11)

In [15]:
# Multi layer preceptron

model = Sequential()
model.add(Dense(32, activation='relu',kernel_regularizer=keras.regularizers.L2(0.03), input_dim=11, kernel_initializer='he_normal'))
#model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu', kernel_regularizer=keras.regularizers.L2(0.03), kernel_initializer='he_normal'))
#model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                384       
                                                                 
 batch_normalization (Batch  (None, 32)                128       
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 batch_normalization_1 (Bat  (None, 32)                128       
 chNormalization)                                                
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 batch_normalization_2 (Bat  (None, 16)                6

In [16]:
model.get_weights()

[array([[ 0.17799364, -0.04649976, -0.4151851 , -0.25007302, -0.4608499 ,
         -0.09991144,  0.2200182 , -0.8686439 , -0.59024817, -0.5391048 ,
         -0.11456428, -0.04013703,  0.6600239 ,  0.3669431 ,  0.05357071,
         -0.51938546,  0.43675587,  0.05880391,  0.19791704,  0.02160454,
          0.10142463,  0.3180619 ,  0.4014701 ,  0.22905579,  0.28730634,
         -0.1499701 , -0.38609082,  0.05044844,  0.66706085, -0.02399244,
          0.1663564 , -0.25776443],
        [-0.1238649 , -0.42157224,  0.57241213,  0.32832226,  0.74736494,
         -0.5546416 ,  0.17521793,  0.81111854,  0.07169497,  0.41348064,
         -0.3101607 , -0.46083698, -0.21912478, -0.36179644, -0.43567052,
         -0.24620645, -0.18939932, -0.03538012, -0.05220845,  0.05228781,
          0.10791662,  0.4827645 ,  0.09149259,  0.43563664,  0.26825845,
         -0.7068193 ,  0.8970696 , -0.12943026,  0.40437794,  0.14573373,
         -0.12718222, -0.92471915],
        [-0.21815887, -0.5048122 ,  0.60

In [17]:

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
history = model.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
374/374 [==============================] - 1s 839us/step - loss: 3.3212 - accuracy: 0.7328 - val_loss: 1.8280 - val_accuracy: 0.7614
Epoch 2/100
374/374 [==============================] - 0s 621us/step - loss: 1.2149 - accuracy: 0.7982 - val_loss: 0.8353 - val_accuracy: 0.8137
Epoch 3/100
374/374 [==============================] - 0s 597us/step - loss: 0.6867 - accuracy: 0.8119 - val_loss: 0.5845 - val_accuracy: 0.8106
Epoch 4/100
374/374 [==============================] - 0s 596us/step - loss: 0.5239 - accuracy: 0.8183 - val_loss: 0.4925 - val_accuracy: 0.8157
Epoch 5/100
374/374 [==============================] - 0s 587us/step - loss: 0.4683 - accuracy: 0.8215 - val_loss: 0.4549 - val_accuracy: 0.8282
Epoch 6/100
374/374 [==============================] - 0s 598us/step - loss: 0.4508 - accuracy: 0.8212 - val_loss: 0.4342 - val_accuracy: 0.8285
Epoch 7/100
374/374 [==============================] - 0s 647us/step - loss: 0.4374 - accuracy: 0.8206 - val_loss: 0.4248 - val_ac

In [19]:
# Remove dropout and batch normalization, etc to check model score improvements
# Multi layer preceptron

# model = Sequential()
# model.add(Dense(32, activation='relu',kernel_regularizer=keras.regularizers.L2(0.03), input_dim=11, kernel_initializer='he_normal'))
# model.add(Dropout(0.5))
# model.add(BatchNormalization())

# model.add(Dense(32, activation='relu', kernel_regularizer=keras.regularizers.L2(0.03), kernel_initializer='he_normal'))
# model.add(Dropout(0.5))
# model.add(BatchNormalization())

# model.add(Dense(1, activation='sigmoid'))

# model.summary()

In [20]:
y_pred = model.predict(x_test)
y_pred

125/125 [==============================] - 0s 300us/step


array([[0.60661435],
       [0.6633061 ],
       [0.8036168 ],
       ...,
       [0.7657363 ],
       [0.18595271],
       [0.46575156]], dtype=float32)

In [21]:
from sklearn.metrics import r2_score
print(r2_score(y_test, y_pred))

0.5182939962486581


In [ ]:
#optimization method

model = Sequential()
model.add(Dense(32, activation='relu',input_dim=11))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', metrics=['accuracy'])
import time
from datetime import datetime, time

In [ ]:
start = time.time()
history = model.fit(x_train, y_train, batch_size=11944, epochs=5, validation_data=(x_test, y_test))

print(time.time() - start)

In [ ]:
#Schostic GD
start = time.time()
history = model.fit(x_train, y_train, batch_size=1, epochs=5, validation_data=(x_test, y_test))

print(time.time() - start)